周内效应——横向分析篇

In [1]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [17]:
# 具体基金
# asset = 'jrj.510310'
# baselineasset = 'jrj.510310'
# baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [3]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


大家好，我是格子衫小C，我们今天继续研究周内效应。

上次主要针对的是沪深300指数，我们发现了较明显的周内效应，今天我们分析一些别的资产。

纳斯达克指数

In [6]:
asset = 'jqdata.513100_XSHG|1d'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='纳斯达克指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

这可以咱们大A股好多了，7年多的时间差不多5倍。

直接看看持有一天的情况。

In [11]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [12]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance
5,纳斯达克指数,0.285680,2020-02-13,2020-03-18,4.559679,2.257154,0.482266,0.200370,0.670394
0,周一买入持有1天,0.164572,2018-07-24,2019-08-19,1.858886,0.937120,0.116362,0.092157,0.021288
2,周三买入持有1天,0.136644,2018-08-02,2018-10-31,1.484818,0.429709,0.065683,0.083041,0.018984
3,周四买入持有1天,0.193122,2020-02-07,2020-09-17,1.477314,0.400363,0.064667,0.086588,0.031545
1,周二买入持有1天,0.149458,2014-12-10,2018-07-17,1.114425,-0.179396,0.015502,0.080814,0.002183
4,周五买入持有1天,0.220514,2019-12-23,2020-04-24,1.054111,-0.238550,0.007331,0.095029,0.008558


In [13]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1)

wrmt[0][['title', 'total']]

,title,total
0,周一买入持有1天,0.548571
1,周二买入持有1天,0.495822
2,周三买入持有1天,0.546218
3,周四买入持有1天,0.529745
4,周五买入持有1天,0.515060


直接看胜率好了，因为纳斯达克指数整体是上涨的，所以胜率几乎都高于50%，但并不太明显（沪深300都有2天接近60%了）。

所以直接看PNL也没啥机会。

标准普尔指数

In [18]:
asset = 'jqdata.513500_XSHG|1d'
asset = 'jrj.096001'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='标准普尔指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

标普没有纳斯达克指数那么好，而且2020年疫情过后涨幅不如纳指，但大体上也算是一个持续上涨趋势了。

In [19]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [20]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance
5,标准普尔指数,0.329404,2020-02-19,2020-03-23,1.808758,0.502185,0.108871,0.157056,0.070592
1,周一买入持有1天,0.069021,2014-12-09,2015-11-02,1.333900,0.231249,0.044948,0.064640,0.006749
4,周四买入持有1天,0.126934,2014-11-21,2016-01-21,1.306915,0.184196,0.041315,0.061432,0.005764
2,周二买入持有1天,0.114313,2020-03-04,2020-04-07,1.300656,0.159088,0.040473,0.065831,0.013586
0,周五买入持有1天,0.239045,2018-09-10,2020-03-27,0.943965,-0.502626,-0.007543,0.074694,0.002582
3,周三买入持有1天,0.201815,2015-10-22,2020-09-23,0.929846,-0.576969,-0.009444,0.068364,0.002356


In [21]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1)

wrmt[0][['title', 'total']]

,title,total
0,周五买入持有1天,0.513158
1,周一买入持有1天,0.512346
2,周二买入持有1天,0.530899
3,周三买入持有1天,0.524496
4,周四买入持有1天,0.564706


胜率上看来，最大也就稍大于55%。

恒生指数

In [22]:
asset = 'jqdata.513600_XSHG|1d'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='恒生指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

恒生指数的数据没有那么长，只找到2015年以后的，这个看起来也不算太好，而且大的走势其实和沪深300很像。

2016、2017上涨，2018下跌，2019年前几个月把19年整年的涨完了。

这个走势大体相似的指数，如果周内效应不明显，也算是可以从另外一个角度验证周内效应。

In [23]:
lstparams = []

for i in range(0, 5):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lststart[i], calcweekday2val2(i + 1, 1)],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lststart[i], 1)],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='{}买入持有{}天'.format(lsttitle[i], 1),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [24]:
dfpnl1b = trdb2py.buildPNLReport(lstpnl1 + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance
5,恒生指数,0.400749,2015-04-08,2016-01-21,1.183143,0.010506,0.032242,0.213362,0.025735
2,周一买入持有1天,0.161153,2015-04-28,2016-02-15,1.086036,-0.188690,0.015146,0.078720,0.003323
3,周五买入持有1天,0.242319,2015-06-01,2016-01-22,1.075530,-0.179996,0.013297,0.092798,0.008733
4,周四买入持有1天,0.173840,2015-10-23,2018-06-28,1.045096,-0.236770,0.007939,0.093175,0.001623
0,周二买入持有1天,0.148919,2016-06-29,2018-09-18,1.015585,-0.283463,0.002744,0.096155,0.001509
1,周三买入持有1天,0.221203,2018-03-15,2020-09-29,0.887099,-0.554565,-0.019876,0.089937,0.005177


In [25]:
wrmt = trdb2py.buildPNLWinRateInYears(lstpnl1)

wrmt[0][['title', 'total']]

,title,total
0,周二买入持有1天,0.465704
1,周三买入持有1天,0.478261
2,周一买入持有1天,0.533333
3,周五买入持有1天,0.533597
4,周四买入持有1天,0.501845


可以看到也都还是在50%上下波动，看起来至少是不明显。

这样看来，咱大A股的周内效应确实是特有的，美股和港股都没有明显的周内效应，特别是港股，大的走势和大A股很像，也不明显。